## <b><font color='darkblue'>Workflow Agents</font></b>
<font size='3ptx'>([source](https://google.github.io/adk-docs/agents/workflow-agents/)) This section introduces "workflow agents" - <b>specialized agents that control the execution flow of its sub-agents</b>.</font>

Workflow agents are specialized components in ADK designed purely for <b>orchestrating the execution flow of sub-agents</b>. Their primary role is to manage how and when other agents run, defining the control flow of a process.

Unlike [**LLM Agents**](https://google.github.io/adk-docs/agents/llm-agents/), which use Large Language Models for dynamic reasoning and decision-making, **Workflow Agents operate based on predefined logic**. They determine the execution sequence according to their type (<font color='brown'>e.g., sequential, parallel, loop</font>) without consulting an LLM for the orchestration itself. This results in <b>deterministic and predictable execution patterns</b>.

ADK provides three core workflow agent types, each implementing a distinct execution pattern:
- <b><font size='3ptx'>Sequential Agents</font></b>: Executes sub-agents one after another, <b>in sequence</b>.
- <b><font size='3ptx'>Loop Agents</font></b>: <b>Repeatedly</b> executes its sub-agents until a specific termination condition is met.
- <b><font size='3ptx'>Parallel Agents</font></b>: Executes multiple sub-agents <b>in parallel</b>.

In [2]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str):
    source_code = !cat $src_path
    source_code = [line.replace('```python', "'''python") for line in source_code]
    source_code = [line.replace('```', "'''") for line in source_code]
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>Why Use Workflow Agents?</font></b>
Workflow agents are essential when you need explicit control over how a series of tasks or agents are executed. They provide:
- <b><font size='3ptx'>Predictability</font></b>: The flow of execution is guaranteed based on the agent type and configuration.
- <b><font size='3ptx'>Reliability</font></b>: Ensures tasks run in the required order or pattern consistently.
- <b><font size='3ptx'>Structure</font></b>: Allows you to build complex processes by composing agents within clear control structures.

While the workflow agent manages the control flow deterministically, the sub-agents it orchestrates can themselves be any type of agent, including intelligent LLM Agent instances. This allows you to combine structured process control with flexible, LLM-powered task execution.

### <b><font color='darkgreen'>Sequential agents</font></b>
<font size='3ptx'><b>The `SequentialAgent` is a workflow agent that executes its sub-agents in the order they are specified in the list.</b></fonb>

Use the `SequentialAgent` when you want the execution to occur in a fixed, strict order.

#### <b>Example</b>
You want to build an agent that can summarize any webpage, using two tools: `Get Page Contents` and `Summarize Page`. Because the agent must always call `Get Page Contents` before calling `Summarize Page` (<font color='brown'>you can't summarize from nothing!</font>), you should build your agent using a <b><font color='blue'>SequentialAgent</font></b>.

As with other [**workflow agents**](https://google.github.io/adk-docs/agents/workflow-agents/), the <b><font color='blue'>SequentialAgent</font></b> is not powered by an LLM, and is thus deterministic in how it executes. That being said, workflow agents are concerned only with their execution (<font color='brown'>i.e. in sequence</font>), and not their internal logic; the tools or sub-agents of a workflow agent may or may not utilize LLMs.

#### <b>How it works</b>
When the <b><font color='blue'>SequentialAgent</font></b>'s `Run Async` method is called, it performs the following actions:
1. <b><font size='3ptx'>Iteration</font></b>: It iterates through the sub agents list in the order they were provided.
2. <b><font size='3ptx'>Sub-Agent Execution</font></b>: For each sub-agent in the list, it calls the sub-agent's `Run Async` method.

![sequential agent flow](https://google.github.io/adk-docs/assets/sequential-agent.png)

#### <b>Full Example: Code Development Pipeline</b>
Consider a simplified code development pipeline:
* <b><font size='3ptx'>Code Writer Agent</font></b>: An LLM Agent that generates initial code based on a specification.
* <b><font size='3ptx'>Code Reviewer Agent</font></b>: An LLM Agent that reviews the generated code for errors, style issues, and adherence to best practices. It receives the output of the Code Writer Agent.
* <b><font size='3ptx'>Code Refactorer Agent</font></b>: An LLM Agent that takes the reviewed code (<font color='brown'>and the reviewer's comments</font>) and refactors it to improve quality and address issues.

A <b><font color='blue'>SequentialAgent</font></b> is perfect for this:
```python
SequentialAgent(sub_agents=[CodeWriterAgent, CodeReviewerAgent, CodeRefactorerAgent])
```

This ensures the code is written, then reviewed, and finally refactored, in a strict, dependable order. <b>The output from each sub-agent is passed to the next by storing them in state via `Output Key`<b/>.
> The <b><font color='blue'>SequentialAgent</font></b> passes the same <b><font color='blue'>InvocationContext</font></b> to each of its sub-agents. This means they all share the same session state, including the temporary (`temp:`) namespace, making it easy to pass data between steps within a single turn.

In [8]:
show_source_code('workflow_agents/sequential_agent_ex1/agent.py')


```python
#!/usr/bin/env python
# https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/#full-example-code-development-pipeline
import asyncio

from google.adk.agents import Agent
from google.adk.agents import LlmAgent
from google.adk.agents import SequentialAgent
from google.adk.tools import FunctionTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

APP_NAME="weather_sentiment_agent"
USER_ID="user1234"
SESSION_ID="1234"
GEMINI_MODEL="gemini-2.0-flash"


# --- 1. Define Sub-Agents for Each Pipeline Stage ---

# Code Writer Agent
# Takes the initial specification (from user query) and writes code.
code_writer_agent = LlmAgent(
    name="CodeWriterAgent",
    model=GEMINI_MODEL,
    # Change 3: Improved instruction
    instruction="""You are a Python Code Generator.
Based *only* on the user's request, write Python code that fulfills the requirement.
Output *only* the complete Python code block, enclosed in triple backticks ('''python ... ''').
Do not add any other text before or after the code block.
""",
    description="Writes initial Python code based on a specification.",
    output_key="generated_code" # Stores output in state['generated_code']
)

# Code Reviewer Agent
# Takes the code generated by the previous agent (read from state) and provides feedback.
code_reviewer_agent = LlmAgent(
    name="CodeReviewerAgent",
    model=GEMINI_MODEL,
    # Change 3: Improved instruction, correctly using state key injection
    instruction="""You are an expert Python Code Reviewer.
    Your task is to provide constructive feedback on the provided code.

    **Code to Review:**
    '''python
    {generated_code}
    '''

**Review Criteria:**
1.  **Correctness:** Does the code work as intended? Are there logic errors?
2.  **Readability:** Is the code clear and easy to understand? Follows PEP 8 style guidelines?
3.  **Efficiency:** Is the code reasonably efficient? Any obvious performance bottlenecks?
4.  **Edge Cases:** Does the code handle potential edge cases or invalid inputs gracefully?
5.  **Best Practices:** Does the code follow common Python best practices?

**Output:**
Provide your feedback as a concise, bulleted list. Focus on the most important points for improvement.
If the code is excellent and requires no changes, simply state: "No major issues found."
Output *only* the review comments or the "No major issues" statement.
""",
    description="Reviews code and provides feedback.",
    output_key="review_comments", # Stores output in state['review_comments']
)

# Code Refactorer Agent
# Takes the original code and the review comments (read from state) and refactors the code.
code_refactorer_agent = LlmAgent(
    name="CodeRefactorerAgent",
    model=GEMINI_MODEL,
    # Change 3: Improved instruction, correctly using state key injection
    instruction="""You are a Python Code Refactoring AI.
Your goal is to improve the given Python code based on the provided review comments.

  **Original Code:**
  '''python
  {generated_code}
  '''

  **Review Comments:**
  {review_comments}

**Task:**
Carefully apply the suggestions from the review comments to refactor the original code.
If the review comments state "No major issues found," return the original code unchanged.
Ensure the final code is complete, functional, and includes necessary imports and docstrings.

**Output:**
Output *only* the final, refactored Python code block, enclosed in triple backticks ('''python ... ''').
Do not add any other text before or after the code block.
""",
    description="Refactors code based on review comments.",
    output_key="refactored_code", # Stores output in state['refactored_code']
)

# --- 2. Create the SequentialAgent ---
# This agent orchestrates the pipeline by running the sub_agents in order.
code_pipeline_agent = SequentialAgent(
    name="CodePipelineAgent",
    sub_agents=[code_writer_agent, code_reviewer_agent, code_refactorer_agent],
    description="Executes a sequence of code writing, reviewing, and refactoring.",
    # The agents will run in the order provided: Writer -> Reviewer -> Refactorer
)

# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = code_pipeline_agent
```

### <b><font color='darkgreen'>Loop agents</font></b>
<font size='3ptx'><b>The `LoopAgent` is a workflow agent that executes its sub-agents in a loop (i.e. iteratively)</b>. It repeatedly runs a sequence of agents for a specified number of iterations or until a termination condition is met.</font>

Use the <b>`LoopAgent`</b> when your workflow involves repetition or iterative refinement, such as revising code.

#### <b>Example</b>
You want to build an agent that can generate images of food, but sometimes when you want to generate a specific number of items (<font color='brown'>e.g. 5 bananas</font>), it generates a different number of those items in the image (e.g. an image of 7 bananas). You have two tools: `Generate Image`, `Count Food Items`.

Because you want to keep generating images until it either correctly generates the specified number of items, or after a certain number of iterations, you should build your agent using a <b>`LoopAgent`</b>.

As with other [**workflow agents**](https://google.github.io/adk-docs/agents/workflow-agents/), the <b>`LoopAgent`</b> is not powered by an LLM, and is thus deterministic in how it executes. That being said, workflow agents are only concerned only with their execution (<font color='brown'>i.e. in a loop</font>), and not their internal logic; the tools or sub-agents of a workflow agent may or may not utilize LLMs.

#### <b>How it Works</b>
When the <b>`LoopAgent`</b>'s `Run Async` method is called, it performs the following actions:
1. <b><font size='3ptx'>Sub-Agent Execution</font></b>: It iterates through the Sub Agents list in order. For each sub-agent, it calls the agent's `Run Async` method.
2. <b><font size='3ptx'>Termination Check</font></b>: Crucially, the LoopAgent itself does not inherently decide when to stop looping. You must implement a termination mechanism to prevent infinite loops. Common strategies include:
     - <b>Max Iterations</b>: Set a maximum number of iterations in the LoopAgent. The loop will terminate after that many iterations.
     - <b>Escalation from sub-agent</b>: Design one or more sub-agents to evaluate a condition (<font color='brown'>e.g., "Is the document quality good enough?", "Has a consensus been reached?"</font>). If the condition is met, the sub-agent can signal termination (<font color='brown'>e.g., by raising a custom event, setting a flag in a shared context, or returning a specific value</font>).

![loop agent flow](https://google.github.io/adk-docs/assets/loop-agent.png)

#### <b>Full Example: Iterative Document Improvement</b>
Imagine a scenario where you want to iteratively improve a document:
* <b><font size='3ptx'>Writer Agent</font></b>: An <b>`LlmAgent`</b> that generates or refines a draft on a topic.
* <b><font size='3ptx'>Critic Agent</font></b>: An <b>`LlmAgent`</b> that critiques the draft, identifying areas for improvement.

```python
LoopAgent(sub_agents=[WriterAgent, CriticAgent], max_iterations=5)
```

In this setup, the <b>`LoopAgent`</b> would manage the iterative process. The <b><font color='blue'>CriticAgent</font></b> could be <b>designed to return a "STOP" signal when the document reaches a satisfactory quality level, preventing further iterations</b>. Alternatively, the max iterations parameter could be used to limit the process to a fixed number of cycles, or external logic could be implemented to make stop decisions.

<b>The loop would run at most five times</b>, ensuring the iterative refinement doesn't continue indefinitely.

In [6]:
show_source_code('workflow_agents/sequential_agent_ex2/agent.py')


```python
# Source:
# Part of agent.py --> Follow https://google.github.io/adk-docs/get-started/quickstart/ to learn the setup

import asyncio
import os
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent, SequentialAgent
from google.genai import types
from google.adk.runners import InMemoryRunner
from google.adk.agents.invocation_context import InvocationContext
from google.adk.tools.tool_context import ToolContext
from typing import AsyncGenerator, Optional
from google.adk.events import Event, EventActions

# --- Constants ---
APP_NAME = "doc_writing_app_v3" # New App Name
USER_ID = "dev_user_01"
SESSION_ID_BASE = "loop_exit_tool_session" # New Base Session ID
GEMINI_MODEL = "gemini-2.0-flash"
STATE_INITIAL_TOPIC = "initial_topic"

# --- State Keys ---
STATE_CURRENT_DOC = "current_document"
STATE_CRITICISM = "criticism"

# Define the exact phrase the Critic should use to signal completion
COMPLETION_PHRASE = "No major issues found."


# --- Tool Definition ---
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the critique indicates no further changes are needed, signaling the iterative process should end."""
  print(f"  [Tool Call] `exit_loop` triggered by {tool_context.agent_name}")
  tool_context.actions.escalate = True
  # Return empty dict as tools should typically return JSON-serializable output
  return {}


# --- Agent Definitions ---

# STEP 1: Initial Writer Agent (Runs ONCE at the beginning)
initial_writer_agent = LlmAgent(
    name="InitialWriterAgent",
    model=GEMINI_MODEL,
    include_contents='none',
    # MODIFIED Instruction: Ask for a slightly more developed start
    instruction=(
        f"""
You are a Creative Writing Assistant tasked with starting a story.
Write the *first draft* of a short story (aim for 2-4 sentences).
Base the content *only* on the topic provided below.
Try to introduce a specific element (like a character, a setting detail, or a starting action) to make it engaging.
    Topic: {{initial_topic}}

Output *only* the story/document text. Do not add introductions or explanations."""),
    description="Writes the initial document draft based on the topic, aiming for some initial substance.",
    output_key=STATE_CURRENT_DOC)


# STEP 2a: Critic Agent (Inside the Refinement Loop)
critic_agent_in_loop = LlmAgent(
    name="CriticAgent",
    model=GEMINI_MODEL,
    include_contents='none',
    # MODIFIED Instruction: More nuanced completion criteria, look for clear improvement paths.
    instruction=f"""
    You are a Constructive Critic AI reviewing a short document draft (typically 2-6 sentences).
    Your goal is balanced feedback.

    **Document to Review:**
    '''
    {{current_document}}
    '''

    **Task:**
    Review the document for clarity, engagement, and basic coherence according to the initial topic (if known).

    IF you identify 1-2 *clear and actionable* ways the document could be improved to better capture the topic or
    enhance reader engagement (e.g., "Needs a stronger opening sentence", "Clarify the character's goal"):
    Provide these specific suggestions concisely. Output *only* the critique text.

    ELSE IF the document is coherent, addresses the topic adequately for its length, and has no glaring errors or obvious omissions:
    Respond *exactly* with the phrase "{COMPLETION_PHRASE}" and nothing else.
    It doesn't need to be perfect, just functionally complete for this stage.
    Avoid suggesting purely subjective stylistic preferences if the core is sound.

    Do not add explanations. Output only the critique OR the exact completion phrase.""",
    description="Reviews the current draft, providing critique if clear improvements are needed, otherwise signals completion.",
    output_key=STATE_CRITICISM)


# STEP 2b: Refiner/Exiter Agent (Inside the Refinement Loop)
refiner_agent_in_loop = LlmAgent(
    name="RefinerAgent",
    model=GEMINI_MODEL,
    # Relies solely on state via placeholders
    include_contents='none',
    instruction=f"""
    You are a Creative Writing Assistant refining a document based on feedback OR exiting the process.
    **Current Document:**
    '''
    {{current_document}}
    '''
    **Critique/Suggestions:**
    {{criticism}}

    **Task:**
    Analyze the 'Critique/Suggestions'.
    IF the critique is *exactly* "{COMPLETION_PHRASE}":
    You MUST call the 'exit_loop' function. Do not output any text.
    ELSE (the critique contains actionable feedback):
    Carefully apply the suggestions to improve the 'Current Document'. Output *only* the refined document text.

    Do not add explanations. Either output the refined document OR call the exit_loop function.""",
    description="Refines the document based on critique, or calls exit_loop if critique indicates completion.",
    tools=[exit_loop], # Provide the exit_loop tool
    output_key=STATE_CURRENT_DOC # Overwrites state['current_document'] with the refined version
)


# STEP 2: Refinement Loop Agent
refinement_loop = LoopAgent(
    name="RefinementLoop",
    # Agent order is crucial: Critique first, then Refine/Exit
    sub_agents=[
        critic_agent_in_loop,
        refiner_agent_in_loop,
    ],
    max_iterations=5 # Limit loops
)

# STEP 3: Overall Sequential Pipeline
# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = SequentialAgent(
    name="IterativeWritingPipeline",
    sub_agents=[
        initial_writer_agent,  # Run first to create initial doc
        refinement_loop,       # Then run the critique/refine loop
    ],
    description="Writes an initial document and then iteratively refines it with critique using an exit tool."
)
```

### <b><font color='darkgreen'>Parallel agents</font></b>
<font size='3ptx'><b>The `ParallelAgent` is a [workflow agent](https://google.github.io/adk-docs/agents/workflow-agents/) that executes its sub-agents concurrently</b>. This dramatically speeds up workflows where tasks can be performed independently.</font>

Use `ParallelAgent` when: For scenarios prioritizing speed and involving independent, resource-intensive tasks, a `ParallelAgent` facilitates efficient parallel execution. <b>When sub-agents operate without dependencies, their tasks can be performed concurrently, significantly reducing overall processing time</b>.

<b>As with other workflow agents, the `ParallelAgent` is not powered by an LLM, and is thus deterministic in how it executes.</b> That being said, workflow agents are only concerned with their execution (<font color='brown'>i.e. executing sub-agents in parallel</font>), and not their internal logic; the tools or sub-agents of a workflow agent may or may not utilize LLMs.

#### <b>Example</b>
<font size='3ptx'><b>This approach is particularly beneficial for operations like multi-source data retrieval or heavy computations, where parallelization yields substantial performance gains</b></font>. Importantly, this strategy assumes no inherent need for shared state or direct information exchange between the concurrently executing agents.

#### <b>How it works</b>
When the `ParallelAgent`'s <font color='blue'>run_async()</font> method is called:
1. <b><font size='3ptx'>Concurrent Execution</font></b>: It initiates the <font color='blue'>run_async()</font> method of each sub-agent present in the `sub_agents` list concurrently. This means all the agents start running at (<font color='brown'>approximately</font>) the same time.
2. <b><font size='3ptx'>Independent Branches</font></b>: Each sub-agent operates in its own execution branch. There is <b>no automatic sharing of conversation history or state between these branches during execution</b>.
3. <b><font size='3ptx'>Result Collection</font></b>: The `ParallelAgent` manages the parallel execution and, typically, provides a way to access the results from each sub-agent after they have completed (<font color='brown'>e.g., through a list of results or events</font>). The order of results may not be deterministic.

#### <b><font size='3ptx'>Independent Execution and State Management</font></b>
<b><font size='3ptx'>It's crucial to understand that sub-agents within a `ParallelAgent` run independently</font></b>. If you need communication or data sharing between these agents, you must implement it explicitly. Possible approaches include:
* <b><font size='3ptx'>Shared `InvocationContext`</font></b>: You could pass a shared <b>`InvocationContext`</b> object to each sub-agent. This object could act as a shared data store. However, you'd need to manage concurrent access to this shared context carefully (<font color='brown'>e.g., using locks</font>) to avoid race conditions.
* <b><font size='3ptx'>External State Management</font></b>: Use an external database, message queue, or other mechanism to manage shared state and facilitate communication between agents.
* <b><font size='3ptx'>Post-Processing</font></b>: Collect results from each branch, and then implement logic to coordinate data afterwards.

![parallel agent flow](https://google.github.io/adk-docs/assets/parallel-agent.png)

#### <b><font size='3ptx'>Full Example: Parallel Web Research</font></b>
Imagine researching multiple topics simultaneously:
1. **Researcher Agent 1**: An `LlmAgent` that researches "renewable energy sources."
2. **Researcher Agent 2**: An `LlmAgent` that researches "electric vehicle technology."
3. **Researcher Agent 3**: An `LlmAgent` that researches "carbon capture methods."

```python
ParallelAgent(sub_agents=[ResearcherAgent1, ResearcherAgent2, ResearcherAgent3])
```

These research tasks are independent. `Using a ParallelAgent` allows them to run concurrently, potentially reducing the total research time significantly compared to running them sequentially. The results from each agent would be collected separately after they finish.

In [3]:
show_source_code('workflow_agents/parallel_agent_ex3/agent.py')


```python
#!/usr/bin/env python
# Part of agent.py --> Follow https://google.github.io/adk-docs/get-started/quickstart/ to learn the setup
import asyncio

from google.adk.agents import Agent
from google.adk.agents import LlmAgent
from google.adk.tools import google_search


GEMINI_MODEL="gemini-2.0-flash"

# --- 1. Define Researcher Sub-Agents (to run in parallel) ---

# Researcher 1: Renewable Energy
researcher_agent_1 = LlmAgent(
    name="RenewableEnergyResearcher",
    model=GEMINI_MODEL,
    instruction="""You are an AI Research Assistant specializing in energy.
 Research the latest advancements in 'renewable energy sources'.
 Use the Google Search tool provided.
 Summarize your key findings concisely (1-2 sentences).
 Output *only* the summary.""",
    description="Researches renewable energy sources.",
    tools=[google_search],
    # Store result in state for the merger agent
    output_key="renewable_energy_result"
)


# Researcher 2: Electric Vehicles
researcher_agent_2 = LlmAgent(
    name="EVResearcher",
    model=GEMINI_MODEL,
    instruction="""You are an AI Research Assistant specializing in transportation.
Research the latest developments in 'electric vehicle technology'.
Use the Google Search tool provided.
Summarize your key findings concisely (1-2 sentences).
Output *only* the summary.""",
    description="Researches electric vehicle technology.",
    tools=[google_search],
    # Store result in state for the merger agent
    output_key="ev_technology_result"
)

# Researcher 3: Carbon Capture
researcher_agent_3 = LlmAgent(
    name="CarbonCaptureResearcher",
    model=GEMINI_MODEL,
    instruction="""You are an AI Research Assistant specializing in climate solutions.
Research the current state of 'carbon capture methods'.
Use the Google Search tool provided.
Summarize your key findings concisely (1-2 sentences).
Output *only* the summary.""",
    description="Researches carbon capture methods.",
    tools=[google_search],
    # Store result in state for the merger agent
    output_key="carbon_capture_result"
)

# --- 2. Create the ParallelAgent (Runs researchers concurrently) ---
# This agent orchestrates the concurrent execution of the researchers.
# It finishes once all researchers have completed and stored their results in state.
parallel_research_agent = ParallelAgent(
    name="ParallelWebResearchAgent",
    sub_agents=[researcher_agent_1, researcher_agent_2, researcher_agent_3],
    description="Runs multiple research agents in parallel to gather information."
)

# --- 3. Define the Merger Agent (Runs *after* the parallel agents) ---
# This agent takes the results stored in the session state by the parallel agents
# and synthesizes them into a single, structured response with attributions.
merger_agent = LlmAgent(
    name="SynthesisAgent",
    model=GEMINI_MODEL,  # Or potentially a more powerful model if needed for synthesis
    instruction="""You are an AI Assistant responsible for combining research findings into a structured report.

 Your primary task is to synthesize the following research summaries, clearly attributing findings to their source areas. Structure your response using headings for each topic. Ensure the report is coherent and integrates the key points smoothly.

 **Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the 'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these specific summaries.**

 **Input Summaries:**

 *   **Renewable Energy:**
     {renewable_energy_result}

 *   **Electric Vehicles:**
     {ev_technology_result}

 *   **Carbon Capture:**
     {carbon_capture_result}

 **Output Format:**

 ## Summary of Recent Sustainable Technology Advancements

 ### Renewable Energy Findings
 (Based on RenewableEnergyResearcher's findings)
 [Synthesize and elaborate *only* on the renewable energy input summary provided above.]

 ### Electric Vehicle Findings
 (Based on EVResearcher's findings)
 [Synthesize and elaborate *only* on the EV input summary provided above.]

 ### Carbon Capture Findings
 (Based on CarbonCaptureResearcher's findings)
 [Synthesize and elaborate *only* on the carbon capture input summary provided above.]

 ### Overall Conclusion
 [Provide a brief (1-2 sentence) concluding statement that connects *only* the findings presented above.]

 Output *only* the structured report following this format. Do not include introductory or concluding phrases outside this structure, and strictly adhere to using only the provided input summary content.
 """,
    description="Combines research findings from parallel agents into a structured, cited report, strictly grounded on provided inputs.",
    # No tools needed for merging
    # No output_key needed here, as its direct response is the final output of the sequence
)


# --- 4. Create the SequentialAgent (Orchestrates the overall flow) ---
# This is the main agent that will be run. It first executes the ParallelAgent
# to populate the state, and then executes the MergerAgent to produce the final output.
sequential_pipeline_agent = SequentialAgent(
    name="ResearchAndSynthesisPipeline",
    # Run parallel research first, then merge
    sub_agents=[parallel_research_agent, merger_agent],
    description="Coordinates parallel research and synthesizes the results."
)

root_agent = sequential_pipeline_agent
```

## <b><font color='darkblue'>Supplement</font></b>
* [GCP Blog - How to build a simple multi-agentic system using Google’s ADK](https://cloud.google.com/blog/products/ai-machine-learning/build-multi-agentic-systems-using-google-adk)